In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, Float, DateTime
from datetime import datetime

In [2]:
def build_DB_URI(db_type, db_lib, user_id, password, db_name,  db_location='localhost', port='5432' ):
    '''
        A method which generates a DB_URI for SQL-Alchemey. Assumption that this will be
        used with Postgresql, however written to be generic.

        arg:

        db_type     --> the type of database, e.g 'postgres', 'mysql'

        db_lib      --> the appropriate sql-alchemy plughin for 
                        db_type, e.g 'psycopg2' or 'pymysql'

        user_id     --> the user name for the database, who has 
                        appropriate permissions

        password    --> the password for the db-user-id.
        db_name     --> the name of the db, e.g. 'esomeprazole'
        db_location --> the address / URL for the database. DEFAULT = localhost
        port        --> the port for the database. DEFAULT = 5432
        
        returns:
        db_URI     --> The URI for SQL-Alchemy of the form:
                       postgres+psycop2://user_id:password@db_location:5432/db_name

    '''
    
    db_URI = db_type+'+'+db_lib+'://'+user_id+':'+password+'@'+db_location+':'+port+'/'+db_name

    return db_URI

In [3]:
db_type = 'postgres'
db_lib = 'psycopg2'
user_id = 'bhima'
password= ''
db_name = 'openfda'

db_URI = build_DB_URI(db_type, db_lib, user_id, password, db_name)
db_engine = create_engine(db_URI, echo=False)
db_engine.connect()
connection= db_engine.connect()

In [4]:
sql_query = 'SELECT * FROM drugs_reactions_countries;'

In [5]:
df = pd.read_sql(sql_query, 
                     con=db_engine)

In [6]:
df.describe()

,safetyreportid,activesubstancename,openfda_generic_name,reactionmeddrapt,primarysource_reportercountry
count,87828606,87828606,87828606,87828606,87828606
unique,237943,351,2250,9613,141
top,9350740,,,Pain,US
freq,3360000,87764046,26877200,725617,72406847


In [7]:
drug_and_raction_grouped = df.groupby(['reactionmeddrapt', 'activesubstancename'])
print(drug_and_raction_grouped.agg(np.size))

                                                     safetyreportid  \
reactionmeddrapt                activesubstancename                   
ACTH-producing pituitary tumour                                  70   
AIDS dementia complex                                            34   
Abasia                                                        56098   
Abdomen crushing                                                 20   
Abdominal abscess                                              3700   
...                                                             ...   
Zinc deficiency                                                  81   
Zygomycosis                                                     490   
pH body fluid abnormal                                          480   
pH body fluid decreased                                        1014   
pH urine increased                                               52   

                                                     openfda_generic_name  \

In [8]:
drug_and_raction_df = df[['activesubstancename', 'reactionmeddrapt']].drop_duplicates()

In [9]:
drug_and_raction_df.describe()

,activesubstancename,reactionmeddrapt
count,12692,12692
unique,351,9613
top,,Pyrexia
freq,9612,72


In [10]:
drug_and_raction_df.groupby(['reactionmeddrapt', 'activesubstancename']).groups

{('ACTH-producing pituitary tumour', ''): Int64Index([7178205], dtype='int64'),
 ('AIDS dementia complex', ''): Int64Index([570732], dtype='int64'),
 ('Abasia', ''): Int64Index([4266], dtype='int64'),
 ('Abdomen crushing', ''): Int64Index([85201058], dtype='int64'),
 ('Abdominal abscess', ''): Int64Index([1492694], dtype='int64'),
 ('Abdominal adhesions', ''): Int64Index([1599829], dtype='int64'),
 ('Abdominal cavity drainage', ''): Int64Index([7034116], dtype='int64'),
 ('Abdominal compartment syndrome', ''): Int64Index([1094352], dtype='int64'),
 ('Abdominal compartment syndrome',
  'BEVACIZUMAB'): Int64Index([1999486], dtype='int64'),
 ('Abdominal compartment syndrome',
  'BLEOMYCIN SULFATE'): Int64Index([1999232], dtype='int64'),
 ('Abdominal compartment syndrome',
  'CARBOPLATIN'): Int64Index([1999184], dtype='int64'),
 ('Abdominal compartment syndrome',
  'CISPLATIN'): Int64Index([1999209], dtype='int64'),
 ('Abdominal compartment syndrome',
  'CYCLOPHOSPHAMIDE'): Int64Index([199

In [20]:
genericname_and_raction_grouped = df.groupby(['activesubstancename', 'openfda_generic_name'])
counted_genericname_and_raction_grouped = genericname_and_raction_grouped.agg(np.size)

In [12]:
genericname_and_raction_df = df[['activesubstancename', 'openfda_generic_name']].drop_duplicates()

In [13]:
genericname_and_raction_df.describe()

,activesubstancename,openfda_generic_name
count,2630,2630
unique,351,2250
top,,
freq,2250,276


In [15]:
genericname_and_raction_df.groupby(['activesubstancename', 'openfda_generic_name']).groups

{('', ''): Int64Index([22], dtype='int64'),
 ('', '(ESTRADIOL TRANSDERMAL SYSTEM)'): Int64Index([2121110], dtype='int64'),
 ('',
  '(SUMATRIPTAN INJECTION) 6MG/0.5ML, AUTO-INJECTOR;SUMATRIPTAN'): Int64Index([10482796], dtype='int64'),
 ('',
  '.ALPHA.-TOCOPHEROL ACETATE, DL-, ASCORBIC ACID, CYANOCOBALAMIN, SODIUM FLUORIDE, FOLIC ACID, NIACIN, PYRIDOXINE, RIBOFLAVIN, THIAMINE, VITAMIN A, AND VITAMIN D;SODIUM FLUORIDE'): Int64Index([11381808], dtype='int64'),
 ('',
  '.BETA.-CAROTENE, ASCORBIC ACID, CHOLECALCIFEROL, .ALPHA.-TOCOPHEROL ACETATE, DL-, THIAMINE MONONITRATE, RIBOFLAVIN, NIACINAMIDE, PYRIDOXINE HYDROCHLORIDE, FOLIC ACID, 5-METHYLTETRAHYDROFOLIC ACID, CALCIUM FORMATE, FERROUS ASPARTO GLYCINATE, CYANOCOBALAMIN, BIOTIN, POTASSIUM IODIDE, MAGNESIUM OXIDE, ZINC OXIDE AND CUPRIC OXIDE;.BETA.-CAROTENE, ASCORBIC ACID, CHOLECALCIFEROL, .ALPHA.-TOCOPHEROL ACETATE, DL-, THIAMINE MONONITRATE, RIBOFLAVIN, NIACINAMIDE, PYRIDOXINE HYDROCHLORIDE, FOLIC ACID, CYANOCOBALAMIN, BIOTIN, CALCIUM PA

In [16]:
genericname_and_raction_grouped = df.groupby(['activesubstancename', 'openfda_generic_name'])
counted_genericname_and_raction = genericname_and_raction_grouped.agg(np.size)

safetyreportid  \
reactionmeddrapt                primarysource_reportercountry                   
ACTH-producing pituitary tumour US                                         70   
AIDS dementia complex           COUNTRY NOT SPECIFIED                      12   
                                JP                                         12   
                                US                                         10   
Abasia                          AE                                         44   
...                                                                       ...   
Zygomycosis                     JP                                         77   
                                US                                        200   
pH body fluid abnormal          US                                        480   
pH body fluid decreased         DE                                       1014   
pH urine increased              IR                                         52   

                                                               activesubstancename  \
reactionmeddrapt                primarysource_reportercountry                        
ACTH-producing pituitary tumour US                                              70   
AIDS dementia complex           COUNTRY NOT SPECIFIED                           12   
                                JP                                              12   
                                US                                              10   
Abasia                          AE                                              44   
...                                                                            ...   
Zygomycosis                     JP                                              77   
                                US                                             200   
pH body fluid abnormal          US                                             480   
pH body fluid decreased         DE                                            1014   
pH urine increased              IR                                              52   

                                                               openfda_generic_name  
reactionmeddrapt                primarysource_reportercountry                        
ACTH-producing pituitary tumour US                                               70  
AIDS dementia complex           COUNTRY NOT SPECIFIED                            12  
                                JP                                               12  
                                US                                               10  
Abasia                          AE                                               44  
...                                                                             ...  
Zygomycosis                     JP                                               77  
                                US                                              200  
pH body fluid abnormal          US                                              480  
pH body fluid decreased         DE                                             1014  
pH urine increased              IR                                               52  

[54110 rows x 3 columns]

In [17]:
country_and_reaction_df = df[['reactionmeddrapt', 'primarysource_reportercountry']].drop_duplicates()

In [18]:

country_and_reaction_df.describe()

,reactionmeddrapt,primarysource_reportercountry
count,54110,54110
unique,9613,141
top,Pyrexia,US
freq,82,7771


In [19]:
country_and_reaction_df.groupby(['reactionmeddrapt', 'primarysource_reportercountry']).groups

{('ACTH-producing pituitary tumour',
  'US'): Int64Index([7178205], dtype='int64'),
 ('AIDS dementia complex',
  'COUNTRY NOT SPECIFIED'): Int64Index([570732], dtype='int64'),
 ('AIDS dementia complex', 'JP'): Int64Index([76050869], dtype='int64'),
 ('AIDS dementia complex', 'US'): Int64Index([6656662], dtype='int64'),
 ('Abasia', 'AE'): Int64Index([78975707], dtype='int64'),
 ('Abasia', 'AR'): Int64Index([830248], dtype='int64'),
 ('Abasia', 'AU'): Int64Index([1721108], dtype='int64'),
 ('Abasia', 'BE'): Int64Index([72820009], dtype='int64'),
 ('Abasia', 'BR'): Int64Index([965606], dtype='int64'),
 ('Abasia', 'CA'): Int64Index([1307971], dtype='int64'),
 ('Abasia', 'CM'): Int64Index([62839480], dtype='int64'),
 ('Abasia', 'CN'): Int64Index([77114388], dtype='int64'),
 ('Abasia', 'CO'): Int64Index([6809291], dtype='int64'),
 ('Abasia', 'COUNTRY NOT SPECIFIED'): Int64Index([4266], dtype='int64'),
 ('Abasia', 'DE'): Int64Index([2330719], dtype='int64'),
 ('Abasia', 'DK'): Int64Index([688

In [21]:
country_and_reaction__grouped = df.groupby(['reactionmeddrapt', 'primarysource_reportercountry'])
counted_genericname_and_raction = country_and_reaction__grouped.agg(np.size)